In [1]:
# Import necessary libraries
# from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

# Load the BigScience Bloom model
model = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')
# model = AutoModelForCausalLM.from_pretrained(
#     "bigscience/bloom",
#     device_map="auto",
#     torch_dtype="auto",
# )

# Define a function to generate response
def generate_response(prompt):
    # Use the model to generate a response
    response = model(prompt, max_length=1000, do_sample=True, temperature=0.7)

    # Convert the response to a string
    response_str = response[0]['generated_text'].split("\n")[0]

    return response_str

# Define the main function to interact with the user
def main():
    # Print a welcome message
    print("Hello! I am a chatbot that can answer questions about California water regulations. How can I help you today?")

    # Keep the chatbot running until the user decides to exit
    while True:
        # Get input from the user
        prompt = input("You: ")

        # If the user wants to exit, break out of the loop
        if prompt.lower() in ['exit', 'quit', 'bye']:
            print("Goodbye!")
            break

        # Generate a response
        response = generate_response(prompt)

        # Print the response
        print("Chatbot: ", response)

# Call the main function
if __name__ == '__main__':
    main()

c:\Users\zachz\Documents\GitHub\model testing\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello! I am a chatbot that can answer questions about California water regulations. How can I help you today?


c:\Users\zachz\Documents\GitHub\model testing\venv\lib\site-packages\transformers\generation\utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [2]:
# Import necessary libraries
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load the BigScience Bloom model
model_name = "EleutherAI/gpt-neo-2.7B"
model = AutoModelForCausalLM.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the dataset
df = pd.read_csv("60.csv")

# Preprocess the data
data = []
for index, row in df.iterrows():
    input_text = row['question']
    output_text = row['answer']
    text = f"Question: {input_text} Answer: {output_text}\n"
    data.append(text)

# Tokenize the data
tokenized_data = tokenizer(data, truncation=True, padding=True)

# Create the TextDataset
text_dataset = TextDataset(tokenized_data['input_ids'], tokenized_data['attention_mask'])

# Create the DataCollator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./california_water_regulations_chatbot',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=16,
    warmup_steps=500,
    learning_rate=1e-4,
    save_total_limit=2,
    logging_steps=100,
    eval_steps=500,
    save_steps=500
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=text_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the finetuned model
model.save_pretrained("california_water_regulations_chatbot")
tokenizer.save_pretrained("california_water_regulations_chatbot")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2


In this example, the california_water_regulations.csv file contains a dataset of questions and answers about California water regulations. The code first preprocesses the data, tokenizes the data, creates a TextDataset, and defines a DataCollator for language modeling. It then defines the training arguments and creates a Trainer object to train the model on the preprocessed dataset. Finally, it saves the finetuned model and tokenizer to a new directory called california_water_regulations_chatbot.

After finetuning the model, you can use the same code as before to create a chatbot that can answer questions about California water regulations, but now with better accuracy and performance. You can also adjust the training parameters and experiment with different datasets to further improve the chatbot's performance.